In [1]:
import numpy as np
from __future__ import print_function
from six.moves import xrange

In [2]:
data = open('input2.txt', 'r').read().split(" ")

In [3]:
chars = list(set(data))

In [4]:
data_size, vocab_size = len(data), len(chars)

In [5]:
char_to_ix = {ch:i for i,ch in enumerate(chars)}
ix_to_char = {i:ch for i,ch in enumerate(chars)}

In [6]:
hidden_size = 100
seq_length = 25
learning_rate = 1e-1

In [7]:
# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

In [44]:
def lossFun(inputs, targets, hprev):
    x, h, y, p = {}, {}, {}, {}
    h[-1] = np.copy(hprev)
    loss = 0
    
    # forward pass
    for t in xrange(len(inputs)):
        #1-hot encoding
        x[t] = np.zeros((vocab_size,1))
        x[t][inputs[t]] = 1
        
        #input and hidden
        h[t] = np.tanh(np.dot(Wxh, x[t]) + np.dot(Whh, h[t-1]) + bh)
        
        #output
        y[t] = np.dot(Why, h[t]) + by 
        p[t] = np.exp(y[t]) / np.sum(np.exp(y[t])) 
        
        #loss
        loss += -np.log(p[t][targets[t],0])
        
  
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    
    dhnext = np.zeros_like(h[0])
    
    for t in reversed(xrange(len(inputs))):
        
        dy = np.copy(p[t])
        dy[targets[t]] -= 1 
        
        dWhy += np.dot(dy, h[t].T)
        
        dby += dy
        
        dh = np.dot(Why.T, dy) + dhnext 
        dhraw = (1 - h[t] * h[t]) * dh 
        
        dbh += dhraw
        
        dWxh += np.dot(dhraw, x[t].T)
        dWhh += np.dot(dhraw, h[t-1].T)
        
        dhnext = np.dot(Whh.T, dhraw)
    
    #clipping for exploding gradients
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) 
    
    return loss, dWxh, dWhh, dWhy, dbh, dby, h[len(inputs)-1]

In [45]:
def sample(h, seed_ix, n):
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in xrange(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

In [48]:
n, p = 0, 0

mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)

mbh, mby = np.zeros_like(bh), np.zeros_like(by) 

smooth_loss = -np.log(1.0/vocab_size)*seq_length 

num_itr = 10000

for i in range(num_itr):
    if p+seq_length+1 >= len(data) or n == 0: 
        hprev = np.zeros((hidden_size,1)) # reset RNN memory
        p = 0 # go from start of data
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

    if n % 100 == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ' '.join(ix_to_char[ix] for ix in sample_ix)
        print ('----\n %s \n----' % (txt, ))

    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 100 == 0: 
        print ('iter %d, loss: %f' % (n, smooth_loss))
  
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

    p += seq_length
    n += 1  

----
 thee.
Go silver; ark; Sarah's Kedemah:
These country.
And kindred.
And lord.
And thee?
Then servant: sister:
And man: sore, foals.
And Abidah, Eldaah. virgin afraid On Hinder womb, hurt, live, milch fled.
So smell stuff? parcel man: gutters, Mahanaim.
And died, gutters, roll sleep.
And void; hindermost.
And Canaan;
And blessed.
And escape.
And stone thee.
Go ghost, me?
And mandrakes kindred.
And flocks: messengers With go: Asher.
And money.
And Perizzites: quiver field: stead, pursue concubines, breaking frost mount.
And household, mouth.
And Pass rods company gone, colts, Jabbok.
And Is whomsoever stones, Rebekah, thee;
That charged overdrive oath chode him.
For toucheth brother;
Two parcel thee;
That Mishma, thee:
I ringstraked, Havilah country.
And whensoever Esau.
And experience sore, maidservants' fled.
So scarce form, blessed: Night. Judith sheep: journey; rams form, Nebajoth; Esau? die.
Let be, me.
For large hire.
So Massa,
Hadar, camest, glory.
And handmaidens household, 

KeyboardInterrupt: 